## 介绍
使用 LlamaIndex 和 NebulaGraph 来构建一个关于费城费利斯队（Philadelphia Phillies）的 RAG 流程

跟着这个教程，你将学会
- LlamaIndex + Azure 的配置方法
- NebulaGraph 服务搭建
- 你讲学会 NebulaGraph 的基本使用
- NebulaGraph 主要的几种查询方式
- NebulaGraph 实战用例及分析

## 提前准备
- Docker Desktop
- Azure OpenAI 或者 GPT-4 （免费的 OpenAPI GPT-3.5 有 token 限制）
- python 版本小于 3.12（本项目使用 3.11.6python 版本）


## 安装了必要的库

除了 LlamaIndex 外，我们还需要安装几个库：
- `ipython-ngql`：可以增强你从 Jupyter Notebook 或 iPython 连接到 NebulaGraph 的能力。
- `nebula3-python`：用于连接和管理 NebulaGraph 数据库。
- `pyvis`：能够使用最少的 Python 代码快速生成可视化网络图。
- `networkx`：用于研究图和网络。
- `youtube_transcript_api`：这是一个 Python API，允许你获取 YouTube 视频的剧本/字幕。
- `openai-whisper`：openAI 提供的语言库

In [30]:
%pip install llama_index==0.9.21 ipython-ngql nebula3-python pyvis networkx youtube_transcript_api llama-hub openai-whisper pydub

Note: you may need to restart the kernel to use updated packages.


### 调整输出日志配置

In [31]:
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## 连接到 NebulaGraph 并设置新空间
[NebulaGraph 安装文档](https://docs.nebula-graph.io/3.6.0/2.quick-start/1.quick-start-workflow/) 

In [32]:


import os
os.environ["GRAPHD_HOST"] = "127.0.0.1"
os.environ["NEBULA_USER"] = "root"
os.environ["NEBULA_PASSWORD"] = "nebula" 
os.environ["NEBULA_ADDRESS"] = "127.0.0.1:9669"  

%reload_ext ngql
connection_string = f"--address {os.environ['GRAPHD_HOST']} --port 9669 --user root --password {os.environ['NEBULA_PASSWORD']}"
%ngql {connection_string}

Connection Pool Created
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)


,Name
0,demo_basketballplayer
1,phillies_rag


In [33]:
%ngql CREATE SPACE IF NOT EXISTS phillies_rag(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)


""


In [34]:
%ngql SHOW SPACES;

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)


,Name
0,demo_basketballplayer
1,phillies_rag


In [35]:
%%ngql
USE phillies_rag;
CREATE TAG IF NOT EXISTS entity(name string);
CREATE EDGE IF NOT EXISTS relationship(relationship string);

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)


""


In [36]:
%ngql CREATE TAG INDEX IF NOT EXISTS entity_index ON entity(name(256));

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)


""


### 构建 NebulaGraphStore

In [37]:
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore

space_name = "phillies_rag"
edge_types, rel_prop_names = ["relationship"], ["relationship"]
tags = ["entity"]

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [38]:

space_name = "phillies_rag"
edge_types, rel_prop_names = ["relationship"], ["relationship"]
tags = ["entity"]

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

## 加载数据并创建 KG 索引

### OpenAI GPT 配置

In [39]:
# import os
# os.environ["OPENAI_API_KEY"] = "sk-xxx"
# from llama_index import (
#     LLMPredictor,
#     ServiceContext,
#     KnowledgeGraphIndex,
#     SimpleDirectoryReader,
# )
# from llama_index.graph_stores import SimpleGraphStore
# from llama_index import download_loader
# from llama_index.llms import OpenAI

# # define LLM
# llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")
# service_context = ServiceContext.from_defaults(llm=llm, chunk_size=512)

### Azure OpenAI 配置

In [40]:
# For Azure OpenAI
from llama_index.llms import AzureOpenAI
from llama_index.embeddings import AzureOpenAIEmbedding
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    KnowledgeGraphIndex,
    ServiceContext,
)

from llama_index.storage.storage_context import StorageContext

import logging
import sys

from IPython.display import Markdown, display

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

api_key = os.environ["AZURE_OPENAI_KEY"]
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = "2023-09-01-preview"

llm = AzureOpenAI(
    model="gpt-35-turbo-16k",
    deployment_name="gpt-35-turbo-16k",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)


service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
)

In [41]:
from llama_index import load_index_from_storage
from llama_hub.youtube_transcript import YoutubeTranscriptReader
from llama_index import download_loader

try:

    storage_context = StorageContext.from_defaults(persist_dir='./storage_graph', graph_store=graph_store)
    kg_index = load_index_from_storage(
        storage_context=storage_context,
        service_context=service_context,
        max_triplets_per_chunk=15,
        space_name=space_name,
        edge_types=edge_types,
        rel_prop_names=rel_prop_names,
        tags=tags,
        verbose=True,
    )
    index_loaded = True
except:
    index_loaded = False

if not index_loaded:
    # 获取在线网站文档数据和视频数据
    WikipediaReader = download_loader("WikipediaReader")
    loader = WikipediaReader()
    wiki_documents = loader.load_data(pages=['Philadelphia Phillies'], auto_suggest=False)
    print(f'Loaded {len(wiki_documents)} documents')

    # 在线获取视频需要验证，我们将视频下载到 graph_data 文件夹
    # youtube_loader = YoutubeTranscriptReader()
    # youtube_documents = youtube_loader.load_data(ytlinks=['https://www.youtube.com/watch?v=k-HTQ8T7oVw'])    
    # print(f'Loaded {len(youtube_documents)} YouTube documents')

    kg_index = KnowledgeGraphIndex.from_documents(
        # documents=wiki_documents + youtube_documents,
        documents=wiki_documents + SimpleDirectoryReader(
            "./graph_data"
        ).load_data() ,
        storage_context=storage_context,
        max_triplets_per_chunk=15,
        service_context=service_context,
        space_name=space_name,
        edge_types=edge_types,
        rel_prop_names=rel_prop_names,
        tags=tags,
        include_embeddings=True,
    )
    
    kg_index.storage_context.persist(persist_dir='./storage_graph')

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


## 运行查询 NebulaGraph

In [42]:
from llama_index.query_engine import KnowledgeGraphQueryEngine
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore
from IPython.display import Markdown, display

# query_engine = KnowledgeGraphQueryEngine(
#     storage_context=storage_context,
#     service_context=service_context,
#     llm=llm,
#     verbose=True,
# )

hybrid_query_engine = kg_index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=3,
    explore_global_knowledge=True,
)

In [43]:
hybrid_query_engine = kg_index.as_query_engine()
response = hybrid_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POS

<b>Bryce Harper is a well-known baseball player who has achieved significant success in his career. He has won the World Series and has been selected as an All-Star multiple times. Harper is also recognized as a one-time silver slugger. He is known for his exceptional skills and is considered one of the top players in Major League Baseball. Additionally, Harper has received lucrative contracts from both the Philadelphia Phillies and the Los Angeles Dodgers, further highlighting his value and talent in the sport.</b>

In [44]:
query_engine = kg_index.as_query_engine()
response = query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POS

<b>Bryce Harper is a well-known baseball player who has achieved significant success in his career. He has won the World Series and has been selected as an All-Star multiple times. Harper is also recognized for his offensive abilities, as he has been awarded the Silver Slugger and has received contracts from both the Philadelphia Phillies and the Los Angeles Dodgers. His performance on the field and his popularity among baseball fans have made him a prominent figure in the sport.</b>

In [45]:
response = query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POS

<b>Bryce Harper is a well-known baseball player who has achieved significant success in his career. He has won the World Series and has been selected as an All-Star multiple times. Harper is also recognized as a one-time silver slugger. He is known for his exceptional skills and is considered one of the top players in Major League Baseball. Additionally, Harper has received lucrative contracts from both the Philadelphia Phillies and the Los Angeles Dodgers, solidifying his status as one of the highest-paid players in the league.</b>

In [46]:
%%ngql
MATCH (p:`entity`)-[e:relationship]->(m:`entity`)
  WHERE p.`entity`.`name` == 'Bryce Harper'
RETURN p, e, m;

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)


,p,e,m


In [47]:
%ng_draw

<class 'pyvis.network.Network'> |N|=0 |E|=0

In [48]:
response = query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POS

<b>The standing ovation that Trey Turner received had a significant impact on his season. It provided him with a tremendous amount of support and love from the fans, which helped him regain his confidence and mental focus. After the standing ovation, Trey Turner had one of his best games in weeks and continued to perform at a high level in the following games. He experienced a significant improvement in his batting average, home runs, and RBIs. The love and support shown by the fans seemed to have lifted the weight off his shoulders and allowed him to play the game with more happiness and relaxation. Overall, the standing ovation played a crucial role in changing Trey Turner's season for the better.</b>

In [49]:
%%ngql 
MATCH (p:`entity`)-[r:`relationship`]->(q:`entity`)
WHERE p.`entity`.`name` == 'Trey Turner' 
RETURN p, r, q;

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)


,p,r,q
0,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@-88459676706300...","(""a World Series"" :entity{name: ""a World Serie..."
1,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@-88459676706300...","(""the World Series"" :entity{name: ""the World S..."
2,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@-86252438187619...","(""nightmare season"" :entity{name: ""nightmare s..."
3,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@-86252438187619...","(""on MLB first and second team"" :entity{name: ..."
4,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@-86252438187619...","(""silver slugger"" :entity{name: ""silver slugge..."
...,...,...,...
65,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@662731478172054...","(""two-time all-star"" :entity{name: ""two-time a..."
66,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@776539795941827...","(""the all-time MLB record"" :entity{name: ""the ..."
67,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@781885191856417...","(""what happened to Trey Turner"" :entity{name: ..."
68,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@796052197484922...","(""all-time MLB record"" :entity{name: ""all-time..."


In [50]:
%ng_draw

<class 'pyvis.network.Network'> |N|=46 |E|=70

In [51]:
response = query_engine.query("Tell me about some of the facts of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POS

<b>The Philadelphia Phillies have signed Trey Turner to a $300 million contract. They also signed Nick Cassianos to a big deal last year, but he struggled in his first season. The team has coaches, staff, and teammates who have full belief in Turner's ability to bounce back.</b>

In [52]:
%%ngql 
MATCH (p:`entity`)-[e:relationship]->(m:`entity`)
  WHERE p.`entity`.`name` == 'Philadelphia Phillies' 
RETURN p, e, m;

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)


,p,e,m


In [53]:
%ng_draw

<class 'pyvis.network.Network'> |N|=0 |E|=0

## 使用矢量索引进行查询

In [54]:
from llama_index import VectorStoreIndex

WikipediaReader = download_loader("WikipediaReader")
loader = WikipediaReader()
wiki_documents = loader.load_data(pages=['Philadelphia Phillies'], auto_suggest=False)
print(f'Loaded {len(wiki_documents)} documents')

# youtube_loader = YoutubeTranscriptReader()
# youtube_documents = youtube_loader.load_data(ytlinks=['https://www.youtube.com/watch?v=k-HTQ8T7oVw'])    
# print(f'Loaded {len(youtube_documents)} YouTube documents')

vector_index = VectorStoreIndex.from_documents(
    # documents=wiki_documents + youtube_documents,
    documents=wiki_documents + SimpleDirectoryReader("./graph_data").load_data() ,
    service_context=service_context,
)
vector_query_engine = vector_index.as_query_engine()

response = vector_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

Loaded 1 documents


/Users/minp/miniconda3/envs/llamaindex_nebulagraph_phillies/lib/python3.11/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/Users/minp/miniconda3/envs/llamaindex_nebulagraph_phillies/lib/python3.11/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POS

<b>Bryce Harper is a professional baseball player who has played for the Philadelphia Phillies since the 2019 season. He was signed to a 13-year, $330 million deal, making it a significant acquisition for the team. Harper has been a standout player for the Phillies and was named the NL Most Valuable Player for the 2021 season. He has been a key contributor to the team's success, including their first winning season since 2011 and their playoff run in 2022, where they reached the World Series.</b>

In [55]:
response = vector_query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POS

<b>The standing ovation Trey Turner received seemed to have a positive impact on his season. After receiving the ovation, Turner had one of his best games in weeks and felt that the hit he got during the game was one of his best swings all year. Following the ovation, Turner's performance improved significantly, with a batting average of .343, an OPS over a thousand, and several hits, home runs, and RBIs in the 18 games that followed. The love and support shown by the fans seemed to have boosted Turner's confidence and helped him get out of his own head, allowing him to play the game more relaxed and happier.</b>

In [56]:
response = vector_query_engine.query("Tell me about some of the facts of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POS

<b>The Philadelphia Phillies are a professional baseball team based in Philadelphia. They compete in Major League Baseball as a member of the National League East Division. The team's home stadium is Citizens Bank Park, located in the South Philadelphia Sports Complex. The Phillies have a history dating back to 1883 and are the oldest, continuous, one-name, one-city franchise in all of professional sports. They have won two World Series championships, eight National League pennants, and have made 15 playoff appearances. The team has played at several stadiums in Philadelphia, including Recreation Park, Baker Bowl, Shibe Park (later renamed Connie Mack Stadium), Veterans Stadium, and now Citizens Bank Park. The Phillies have a loyal fan base and have had both successes and challenges throughout their history.</b>

## 自定义组合查询引擎（KG 检索器和向量索引检索器的组合）

In [57]:
from llama_index import QueryBundle
from llama_index.schema import NodeWithScore
from llama_index.retrievers import BaseRetriever, VectorIndexRetriever, KGTableRetriever
from typing import List

class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both Vector search and Knowledge Graph search"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        kg_retriever: KGTableRetriever,
        mode: str = "OR",
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._kg_retriever = kg_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        kg_nodes = self._kg_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        kg_ids = {n.node.node_id for n in kg_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in kg_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(kg_ids)
        else:
            retrieve_ids = vector_ids.union(kg_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

In [58]:
from llama_index import get_response_synthesizer
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.retrievers import VectorIndexRetriever, KGTableRetriever

# create custom retriever
vector_retriever = VectorIndexRetriever(index=vector_index)
kg_retriever = KGTableRetriever(
    index=kg_index, retriever_mode="keyword", include_text=False
)
custom_retriever = CustomRetriever(vector_retriever, kg_retriever)

# create response synthesizer
response_synthesizer = get_response_synthesizer(
    service_context=service_context,
    response_mode="tree_summarize",
)

## 探索七种查询方法

In [59]:
# KG vector-based entity retrieval
kg_query_engine = kg_index.as_query_engine()

# KG keyword-based entity retrieval
kg_keyword_query_engine = kg_index.as_query_engine(
    # setting to false uses the raw triplets instead of adding the text from the corresponding nodes
    include_text=False,
    retriever_mode="keyword",
    response_mode="tree_summarize",
)

# KG hybrid entity retrieval
kg_hybrid_query_engine = kg_index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=3,
    explore_global_knowledge=True,
)

# Raw vector index retrieval
vector_query_engine = vector_index.as_query_engine()

# Custom combo query engine
custom_query_engine = RetrieverQueryEngine(
    retriever=custom_retriever,
    response_synthesizer=response_synthesizer,
)

# using KnowledgeGraphQueryEngine
from llama_index.query_engine import KnowledgeGraphQueryEngine

kgqe_query_engine = KnowledgeGraphQueryEngine(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm,
    verbose=True,
)

# using KnowledgeGraphRAGRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.retrievers import KnowledgeGraphRAGRetriever

graph_rag_retriever = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm,
    verbose=True,
)

kg_rag_query_engine = RetrieverQueryEngine.from_args(
    graph_rag_retriever, service_context=service_context
)

In [60]:
response = kg_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POS

<b>Bryce Harper is a well-known baseball player who has achieved significant success in his career. He has won the World Series, been a two-time All-Star, and has received contracts from both the Philadelphia Phillies and the Los Angeles Dodgers. Harper is highly regarded by baseball fans and is considered one of the top players in the sport.</b>

In [61]:
response = kg_keyword_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POS

<b>Bryce Harper is a well-known figure, but without any specific relationships mentioned in the context information, it is difficult to provide a detailed answer about him. However, Bryce Harper is a professional baseball player who has gained recognition for his skills and achievements in the sport. He has played for the Washington Nationals and the Philadelphia Phillies in Major League Baseball. Harper is known for his powerful hitting, speed, and overall athleticism. He has been named an All-Star multiple times and has won various awards throughout his career.</b>

In [62]:
response = kg_hybrid_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POS

<b>I'm sorry, but there is no information provided about Bryce Harper in the given context.</b>

In [63]:
response = vector_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POS

<b>Bryce Harper is a professional baseball player who has played for the Philadelphia Phillies since the 2019 season. He was signed to a 13-year, $330 million deal, making him one of the highest-paid players in the league. Harper has been a key player for the Phillies and has had several notable achievements, including winning the NL Most Valuable Player Award for the 2021 season. He has been a significant contributor to the team's success, helping them reach the playoffs in the 2022 season and advancing to the World Series.</b>

In [64]:
response = custom_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POS

<b>Bryce Harper is a professional baseball player who has played for the Philadelphia Phillies since the 2019 season. He was signed to a 13-year, $330 million deal, which was a significant move for the Phillies during the offseason. Harper has been a standout player for the team and was named the NL Most Valuable Player for the 2021 season. He has been a key part of the Phillies' lineup and has helped improve the team's performance. Harper's contributions have been recognized, and he has played a crucial role in the Phillies' success, including their playoff run in the 2022 season.</b>

In [65]:
response = kgqe_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POS

<b>Bryce Harper is not mentioned in the graph database.</b>

In [66]:
response = kg_rag_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POS

<b>Empty Response</b>

In [67]:
response = kg_query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POS

<b>The standing ovation that Trey Turner received had a significant impact on his season. It provided him with a tremendous amount of support and love from the fans, which helped him regain his confidence and mental focus. After the standing ovation, Trey Turner had one of his best games in weeks and continued to perform at a high level in the following games. He experienced a significant improvement in his batting average, home runs, and RBIs. The love and support shown by the fans seemed to have lifted the weight off his shoulders and allowed him to play the game with more happiness and relaxation. Overall, the standing ovation played a crucial role in changing Trey Turner's season and helping him overcome his struggles.</b>

In [68]:
response = kg_keyword_query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POS

<b>The standing ovation Trey Turner received likely had a positive impact on his season. It may have boosted his confidence and morale, leading to improved performance on the field. The support and appreciation from the fans could have motivated him to overcome any challenges or struggles he was facing, potentially resulting in a turnaround in his season.</b>

In [69]:
response = kg_hybrid_query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POS

<b>The standing ovation that Trey Turner received had a significant impact on his season. It provided him with a boost of confidence and support from the fans, which helped him overcome his struggles. After the standing ovation, Trey Turner had one of his best games in weeks and his performance improved significantly. In the 18 games since the standing ovation, he has been batting .343 with a batting average, an OPS over a thousand, and impressive stats in hits, home runs, and RBIs. The love and support shown by the fans through the standing ovation seemed to have changed Trey Turner's mindset and helped him regain his form.</b>

In [70]:
response = vector_query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POS

<b>The standing ovation Trey Turner received seemed to have a positive impact on his season. After receiving the ovation, Turner had one of his best games in weeks and felt that the hit he got during that game was one of his best swings all year. Following the ovation, Turner's performance improved significantly, with a batting average of .343, an OPS over 1.000, and several hits, home runs, and RBIs in the subsequent 18 games. The love and support shown by the fans seemed to have lifted Turner's spirits and helped him regain confidence, leading to a turnaround in his performance.</b>

In [71]:
response = custom_query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POS

<b>The standing ovation that Trey Turner received had a positive impact on his season. After receiving the ovation, Trey seemed happier, more relaxed, and out of his own head. He had one of his best games in weeks and felt that the hit he got during the game was one of his best swings all year. In the 18 games since the standing ovation, Trey has been performing exceptionally well, with a batting average of .343, an OPS over a thousand, and impressive stats in hits, home runs, and RBIs. It seems that the love and support shown by the fans through the standing ovation had a significant effect on Trey's performance and helped him bounce back from his struggles.</b>

In [72]:
response = kgqe_query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POS

RetryError: RetryError[<Future at 0x317f75710 state=finished raised ValueError>]

In [ ]:
response = kg_rag_query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
Entities processed: ['season', 'ovation', 'standing', 'Turner', 'Trey', 'Trey Turner', 'change', 'standing ovation']
INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.az

<b>The standing ovation Trey Turner received did not change his season.</b>

In [ ]:
response = kg_query_engine.query("Tell me some facts about the current stadium of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:l

<b>I'm sorry, but I cannot provide any facts about the current stadium of the Philadelphia Phillies based on the given context information.</b>

In [ ]:
response = kg_keyword_query_engine.query("Tell me some facts about the current stadium of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:llama_index.indices.knowledge_graph.retrievers:> No relationships found, returning nodes found by keywords.
> No relationships found, returning nodes found by keywords.
> No relationships found, returning nodes found by keywords.
INFO:llama_index.indices.knowledge_graph.retrievers:> No nodes found by keywords, returning empty response.
> No nodes found by keywords, returning empty response.
> No nodes found by keywords, returning empty response.
INFO:httpx:HTTP Request: POST https://firstaipro

<b>The current stadium of the Philadelphia Phillies is a popular venue for baseball games. It is known for its modern facilities and amenities, providing a great experience for fans. The stadium has a capacity to hold a large number of spectators, allowing for a lively and energetic atmosphere during games. It is also equipped with state-of-the-art technology, including video screens and sound systems, enhancing the overall game-watching experience. The stadium is conveniently located, making it easily accessible for fans attending games.</b>

In [ ]:
response = kg_hybrid_query_engine.query("Tell me some facts about the current stadium of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:l

<b>I'm sorry, but the given context does not provide any information about the current stadium of the Philadelphia Phillies.</b>

In [ ]:
response = vector_query_engine.query("Tell me some facts about the current stadium of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"


<b>The current stadium of the Philadelphia Phillies is Citizens Bank Park. It has been the team's home stadium since 2004 and is located in the South Philadelphia Sports Complex.</b>

In [ ]:
response = custom_query_engine.query("Tell me some facts about the current stadium of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:l

<b>The current stadium of the Philadelphia Phillies is Citizens Bank Park. It has been the team's home stadium since 2004 and is located in the South Philadelphia Sports Complex.</b>

In [ ]:
response = kgqe_query_engine.query("Tell me some facts about the current stadium of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
Graph Store Query:
MATCH (s:`entity`)-[:`relationship`]->(p:`entity`)
WHERE s.`name` == 'Philadelphia Phillies'
RETURN p.`name`
Graph Store Response:
{'p.name': []}
INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTT

<b>I'm sorry, but I couldn't find any facts about the current stadium of the Philadelphia Phillies based on the given context information.</b>

In [ ]:
response = kg_rag_query_engine.query("Tell me some facts about the current stadium of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
Entities processed: ['facts', 'current', 'current stadium', 'Philadelphia', 'Philadelphia Phillies', 'stadium', 'Phillies']
INFO:httpx:HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://firstaiproject.op

<b>Empty Response</b>